In [36]:
import midi
import csv

class Estado:
    track = ''
    instrumento = ''
    tempos = []

class Tempo:
    tick_inicial = 'nao informado'
    ms_per_tick = 'nao informado'
    tick_final = 'nao informado'

estado = Estado()
notas_sendo_pressionadas = {}
linhas_csv = []
midi_file = midi.read_midifile("file.mid")
midi_file.make_ticks_abs()


def nota_pressionada(nota):
    pitch = nota.pitch
    notas_sendo_pressionadas[pitch] = nota

def imprimir_informacoes_notas(nota_passada, nota_atual):
    quando_nota_passada_foi_pressionada = converter_tick_para_segundos(nota_passada.tick)
    tempo_atual = converter_tick_para_segundos(nota_atual.tick)
    duracao = tempo_atual - quando_nota_passada_foi_pressionada
    # adaptar o valor da velocidade? velocidade eh a forca que se utiliza determinada nota
    velocidade = nota_atual.data[1] / 128
    # porcentual de velocidade
    canal = nota_atual.channel
    with open('esperanca.csv', 'w') as arquivo:
        writer = csv.writer(arquivo)
        header = ['Start', 'Duration', 'Pitch', 'Velocity', 'Instrument']
        writer.writerow(header)
        
        
    instrumento = f'{canal}-{estado.instrumento}-{estado.track}'
    linha = [quando_nota_passada_foi_pressionada, duracao, nota_atual.pitch, velocidade, instrumento]
    linhas_csv.append(linha)
    
def nota_levantada(nota):
    pitch = nota.pitch
    nota_pressionada = notas_sendo_pressionadas[pitch]
    del notas_sendo_pressionadas[pitch]
    velocity = nota.data[1]
    imprimir_informacoes_notas(nota_pressionada, nota)

def instrumento_selecionado(evento):
    estado.instrumento = evento.text

def track_selecionado(evento):
    estado.track = evento.text
    
def set_tempo(evento):
    mpqn = evento.get_mpqn()
    mpqn = mpqn / 1000
    resolution = midi_file.resolution
    ms_per_tick = mpqn / resolution
    tempo = Tempo()
    tempo.tick_inicial = evento.tick
    tempo.ms_per_tick = ms_per_tick
    if(len(estado.tempos) == 0):
        estado.tempos.append(tempo)
    else:
        tempo_passado = estado.tempos.pop()
        tempo_passado.tick_final = tempo.tick_inicial - 1
        estado.tempos.append(tempo_passado)
        estado.tempos.append(tempo)

def converter_tick_para_segundos(tick):
    tempos = estado.tempos
    tempo_total = 0
    ultimo_tempo = tempos[0]
    
    for tempo in tempos:
        tick_inicial = tempo.tick_inicial
        tick_final = tempo.tick_final
        if tick_final == 'nao informado' or tick <= tick_final:
            tempo_total += (tick - tick_inicial) * tempo.ms_per_tick
            return tempo_total / 1000
        elif tick > tick_final:
            delta_tempo = tick_final - tick_inicial
            tempo_total += (delta_tempo) * tempo.ms_per_tick
    
    
for part in midi_file:
    for event in part:
        if event.name == "Note On":
            nota_pressionada(event)
        elif event.name == "Note Off":
            nota_levantada(event)
        elif event.name == "Track Name":
            track_selecionado(event)
        elif event.name == "Instrument Name":
            instrumento_selecionado(event)
        elif event.name == "Set Tempo":
            set_tempo(event)
            
with open('esperanca.csv', 'w', newline='') as arquivo:
    writer = csv.writer(arquivo)
    header = ['Start', 'Duration', 'Pitch', 'Velocity', 'Instrument']
    writer.writerow(header)
    for linha in linhas_csv:
        writer.writerow(linha)